# Лабараторная работы №3 KNeighborsRegressor
class sklearn.neighbors.KNeighborsRegressor(n_neighbors=5, *, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None, **kwargs)

class sklearn.neighbors.RadiusNeighborsRegressor(radius=1.0, *, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None, **kwargs)

Регрессия на основе соседей может использоваться в случаях, когда метки данных являются непрерывными, а не дискретными переменными. Метка, присвоенная точке запроса, вычисляется на основе среднего значения меток ее ближайших соседей.

scikit-learn реализует два разных соседних регрессора: KNeighborsRegressor реализует обучение на основе ближайшие соседи каждой точки запроса, где - целочисленное значение, указанное пользователем. RadiusNeighborsRegressor реализует обучение на основе соседей в фиксированном радиусе точки запроса, где  - это значение с плавающей запятой, указанное пользователем.

<p style="align: center;"><img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_regression_001.png" width="400"></p>



In [43]:
#импортируем библиотеки
import pandas as pd
import numpy as np
import sklearn
import matplotlib
from matplotlib import pyplot as plt
lib = {'scikit-learn' : sklearn, 'pandas' : pd, 'numpy' : np, 'matplotlib' : matplotlib}
print('Корректность проверена на Python 3.8.5 :')
[(print(f'{i} version :{lib[i].__version__}')) for i in lib]
import warnings
warnings.filterwarnings('ignore')

Корректность проверена на Python 3.8.5 :
scikit-learn version :0.23.2
pandas version :1.1.2
numpy version :1.19.2
matplotlib version :3.3.2


### Загрузим данные

[Скачать данные](http://www3.dsi.uminho.pt/pcortez/forestfires/).   
Или можно загрузить их из библиотеки sklearn

In [44]:
df = pd.read_csv('forestfires.csv')
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [45]:
df.describe()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,4.669246,4.299807,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292
std,2.313778,1.229900,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818
min,1.000000,2.000000,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000
25%,3.000000,4.000000,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000
50%,4.000000,4.000000,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000
75%,7.000000,5.000000,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000
max,9.000000,9.000000,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000


In [46]:
X = df.loc[:, :'rain']
y = pd.DataFrame(df.area)

In [47]:
print(X["month"].value_counts())
print(X['day'].value_counts())

aug    184
sep    172
mar     54
jul     32
feb     20
jun     17
oct     15
dec      9
apr      9
jan      2
may      2
nov      1
Name: month, dtype: int64
sun    95
fri    85
sat    84
mon    74
tue    64
thu    61
wed    54
Name: day, dtype: int64


In [48]:
X["month"] = df.month.replace({"jan": 1, "feb":2, "mar":3, "apr":4,"may":5,"jun":6,
                               "jul":7,"aug":8,"sep":9,"oct":10,"nov":11,"dec":12,})

X['day'] = df.day.replace({'mon':1, 'tue':2, 'wed':3, 'thu':4, 'fri':5, 'sat':6, 'sun':7})
X.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain
0,7,5,3,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0
1,7,4,10,2,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0
2,7,4,10,6,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0
3,8,6,3,5,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2
4,8,6,3,7,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0


### KNeighborsRegressor

In [49]:
%%time
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

grid_params = {
    'n_neighbors': range(1,50,5),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'metric': ['euclidean','manhattan', 'chebyshev', 'minkowski'],
    'n_jobs' : [-1]
    
}

gs_knn = GridSearchCV(KNeighborsRegressor(), grid_params, scoring='neg_mean_squared_error', cv=5)
gs_knn.fit(X_train, y_train)

print(f'Best score MSE : {-(gs_knn.best_score_)}')
print(f'Best parametrs : {gs_knn.best_params_}')

Best score MSE : 2097.382609521649
Best parametrs : {'algorithm': 'ball_tree', 'metric': 'manhattan', 'n_jobs': -1, 'n_neighbors': 46, 'weights': 'uniform'}
CPU times: user 27.9 s, sys: 26.8 s, total: 54.8 s
Wall time: 19.5 s


Так как weights='uniform' используются по умолчанию, то остается указать n_neighbors=46, metric='manhattan', n_jobs=-1 при обучении модели.

In [57]:
regressor = KNeighborsRegressor(n_neighbors=46, algorithm='ball_tree', metric='manhattan', n_jobs=-1) 
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

23.138882943143813

In [58]:
print(np.array(y_test[:20]).flatten())
print(y_pred[:20].flatten())

[27.35  0.    2.18 88.49  2.69  0.    1.94  0.   16.33  0.    2.47 22.03
 10.08  0.   18.3   0.    1.56 43.32  1.1   7.36]
[ 5.33543478 14.50413043  6.9373913  18.59369565  1.85608696 15.24630435
  6.5273913   4.33108696  6.6673913   6.10065217 13.99195652 14.53304348
 14.53304348  5.2676087   7.95978261  9.28456522  6.43152174  8.635
  6.29978261 10.89326087]


## Scaling

In [52]:
from sklearn.preprocessing import StandardScaler

z = (x - u) / s

where u is the mean of the training samples or zero if with_mean=False, and s is the standard deviation of the training samples or one if with_std=False.

In [59]:
#создаем стандартный scaler
scaler = StandardScaler()
scaler.fit(X_train, y_train)
scaled_train_data = scaler.transform(X_train)
scaled_test_data = scaler.transform(X_test)

In [60]:

regressor.fit(scaled_train_data, y_train)
metrics.mean_absolute_error(y_test, regressor.predict(scaled_test_data))

23.203040133779265

In [61]:
print(np.array(y_test[:20]).flatten())

[27.35  0.    2.18 88.49  2.69  0.    1.94  0.   16.33  0.    2.47 22.03
 10.08  0.   18.3   0.    1.56 43.32  1.1   7.36]


In [62]:
print(regressor_scal.predict(scaled_test_data)[:10].flatten())

[11.81804348  3.7223913   8.84347826  9.0676087  16.38195652  5.55108696
  6.6573913   5.72456522 11.37565217  6.28043478]


### RadiusNeighborsRegressor

In [63]:
%%time
from sklearn.neighbors import RadiusNeighborsRegressor 

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)


grid_params = {
    'radius': range(100, 1120, 50),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    #'metric': ['euclidean','manhattan', 'chebyshev', 'minkowski'],
    'n_jobs' : [-1]   
}

gs_radius_knn = GridSearchCV(RadiusNeighborsRegressor(), grid_params,
                      scoring='neg_mean_squared_error',
                      cv=5)
gs_radius_knn.fit(X_train, y_train)

print(f'Best score MSE : {-(gs_knn.best_score_)}')
print(f'Best parametrs : {gs_knn.best_params_}')

Best score MSE : 2097.382609521649
Best parametrs : {'algorithm': 'ball_tree', 'metric': 'manhattan', 'n_jobs': -1, 'n_neighbors': 46, 'weights': 'uniform'}
CPU times: user 28.9 s, sys: 42.2 s, total: 1min 11s
Wall time: 15.8 s


In [64]:
regressor2 = RadiusNeighborsRegressor(radius=900, weights='distance', algorithm='brute', n_jobs=-1).fit(X_train, y_train)

In [65]:
y_pred = regressor2.predict(X_test)

In [72]:
mean_absolute_error(y_test, y_pred)

23.019862370066065

In [67]:
print(np.array(y_test[:20]).flatten())
print(y_pred[:20].flatten())

[27.35  0.    2.18 88.49  2.69  0.    1.94  0.   16.33  0.    2.47 22.03
 10.08  0.   18.3   0.    1.56 43.32  1.1   7.36]
[10.51632607 11.17266085  9.70805317 11.46566941  8.83388232 12.63882133
  7.60118198  8.63920716 12.28563756  7.4234696  10.66878151 13.81
 10.16922609  7.85449312 10.41747849  9.38570253  9.56157365 10.26720474
  8.08754803 15.55907803]


## Scaling

In [70]:
regressor2.fit(scaled_train_data, y_train)
mean_absolute_error(y_test, regressor2.predict(scaled_test_data))

23.245092480436938

In [71]:
print(np.array(y_test[:20]).flatten())
print(regressor2.predict(scaled_test_data)[:10].flatten())

[27.35  0.    2.18 88.49  2.69  0.    1.94  0.   16.33  0.    2.47 22.03
 10.08  0.   18.3   0.    1.56 43.32  1.1   7.36]
[ 9.95197333  9.73741425 11.06557436 10.51170389 10.79519132 11.14652543
 10.01823722 10.06078055 10.75466402  9.59005886]
